In [40]:
#clear all varaibles
#%reset

Once deleted, variables cannot be recovered. Proceed (y/[n])? y


In [45]:
import numpy as np
from scipy import stats
# Import curve fitting package from scipy
from scipy.optimize import curve_fit
import matplotlib.pyplot as plt
plt.rcParams['figure.dpi'] = 200
from tqdm import tqdm
#from fbm import fbm, fgn
# from matplotlib import rc
#import colorama

plt.style.use('ggplot')  # Set plot style

In [ ]:
#for debugging
#from IPython.core.debugger import set_trace

In [5]:
def cross_time(data,y_line):
    """
    Calculates the time between crossing a horizontal line: tau's

    Parameters
    ----------
    data : 
        Data that needs to find a crossing
    
    y_line :
        Horizontal line that is crossed.

    Returns
    ----------
    location :
        Location of crossings. This could also serve as a trajectory.
    """
    
    data = np.array(data) #Make sure data is a numpy array. 
    
    #Now see if you have a list or a matrix.
    if len(np.shape(data)) < 2: #The shape of a matrix has 2 numbers.
        data = np.array([data]) #Make array of arrays.
        
    #need to make the y-line at zero
    data = np.sign(data - y_line) #turns data to 0's, -1's, and 1' 
        
    xt, yt = np.where(data == 0) #Find where the 0's are.
    
    #Need to replace all the zeroes.
    for i in range(len(list(zip(xt,yt)))): #Step through each location of 0 in data
        #Inner array == 0 and the rest of array is equal to all 0's
        if yt[i] == 0 and np.array_equal(np.zeros(len(data[xt[i]])), data[xt[i]]):
            data[xt[i]] = np.ones(len(data[xt[i]])) #Turn into all 1's
        
        elif yt[i] == 0: #Found 0 at beginning of array.
            snz = np.nonzero(data[xt[i]])[0][0] #Find the nonzero.
            data[xt[i],yt[i]] = data[xt[i],snz] #Set the 0 to this nonzero number.
        
        elif data[xt[i],yt[i]] == 0: #Found a zero.
            data[xt[i],yt[i]] = data[xt[i],yt[i]-1] #Replace it with the number before.
        
    data = np.add(data[:,:-1],data[:,1:]) #Add i and i+1 for all of array.
    #Crossing now occur at zero.
            
    xl, yl = np.where(data == 0) #this will store where the cross was
     
    return xl, yl+1 #Add 1 for crossing to be after crossing.

In [2]:
def bin_it(series, plot=True):
    
    """
    Makes a histogram of series
    This assumes the data is in integers
    Plots results in lin-lin, log-lin, and log-log.

    Parameters
    ----------
    series :
        Series that must be placed into a histogram.
    plot :
        Boolean for if you want to see the plot.

    Returns
    ----------
    bins : 
        Linear X values of the histogram points. This is the bin of the integer.
        
    counts :
        Linear Y values of the histogram points. This is the count for the bin.
    """
    series = np.array(series)
    
    val = np.array(range(series.max()+1)) #all the values for histogram
    bin_counts = np.bincount(series) #counts for histogram
    
    bins = val[series.min():]
    counts = bin_counts[series.min():]
    
    value_locations = np.where(counts != 0)
        
    
    if plot == True:
    
        fig, (ax1, ax2, ax3) = plt.subplots(1,3, figsize=(9,3))
        ax1.plot(bins, counts)
        ax1.set(title = 'Lin-Lin', xlabel = 'bins', ylabel = 'counts')


        ax2.plot(bins[value_locations], np.log(counts[value_locations]))
        ax2.set(title = 'Log-Lin', xlabel = 'bins', ylabel = 'ln(counts)')

        ax3.plot(np.log(bins[value_locations]), np.log(counts[value_locations]))
        ax3.set(title = 'Log-Log', xlabel = 'ln(bins)', ylabel = 'ln(counts)')

        fig.suptitle('Histograms', fontsize = 20)

        plt.tight_layout()
        fig.subplots_adjust(top=0.8)
        plt.show()
    
    return [bins, counts], [bins[value_locations], counts[value_locations]]

In [2]:
def relax(data, which_plot = "log-lin"):
    
    #When does it cross zero (relaxation)
    first_neg = np.where(data <= 0)[0][0]
    #first_zero = list(data_no_neg).index(0)
    relax = data[:first_neg]
    t = range(len(relax))
    log_t = np.log(range(1,len(relax)))

    fig, (ax1, ax2, ax3) = plt.subplots(1,3, figsize=(9,3))
    ax1.plot(data)
    ax1.set(title = 'Data', xlabel = 't', ylabel = 'y')


    ax2.plot(np.log(relax))
    ax2.set(title = 'Log-Lin', xlabel = 't', ylabel = 'ln(R)')

    ax3.plot(log_t, np.log(relax[1:]))
    ax3.set(title = 'Log-Log', xlabel = 'ln(t)', ylabel = 'ln(R)')

    #ax1.plot(relax)
    #ax1.set(title = 'Relaxation Region', xlabel = 't', ylabel = 'y')

    #first_neg = np.where(np.log(relax) <= 0)[0][0]

    #coefficients = np.polyfit(t[:first_neg], np.log(relax[:first_neg]), 1)

     #= np.array(, t[-1]])

    #ax5.plot(np.log(relax[:first_neg]))
    #ax5.plot(t, (t*coefficients[0]) + coefficients[1], linestyle='--', marker='.', label='$m = {}$'.format(np.round(coefficients[0],4)))
    #ax5.set(title = 'Log-lin', xlabel = 't', ylabel = 'ln(y)')
    #ax5.legend()

    #coefficients = np.polyfit(log_t[:first_neg], np.log(relax[:first_neg]), 1)

    #t=np.array([log_t[0],log_t[-1]])

    #ax6.plot(log_t[:first_neg], np.log(relax[:first_neg]))
    #ax6.plot(t, (t*coefficients[0]) + coefficients[1], linestyle='--', marker='.', label='$m = {}$'.format(np.round(coefficients[0],4)))
    #ax6.set(title = 'Log-Log', xlabel = 'ln(t)', ylabel = 'ln(y)')
    #ax6.legend()

    fig.suptitle('Relaxation', fontsize = 20)

    plt.tight_layout()
    fig.subplots_adjust(top=0.8)
    plt.show()
    
    if which_plot == "log-lin":
        return t, np.log(relax)
    else:
        return log_t, np.log(relax[1:])

In [59]:
def general_linear_fit(x_to_fit, y_to_fit, start, stop, xlabel="x",ylabel='y'):
    
    #Make sure you have enough data points.

    #find closes values
    start = np.abs(np.array(x_to_fit) - start).argmin()
    stop = np.abs(np.array(x_to_fit) - stop).argmin()

    x_to_fit_ends = np.array([x_to_fit[start], x_to_fit[stop]])


    coefficients, V = np.polyfit(x_to_fit[start:stop], y_to_fit[start:stop], 1, cov=True)
    m = coefficients[0]
    m_pm = np.sqrt(V[0][0])
    b = coefficients[1]
    b_pm = np.sqrt(V[1][1])
    
    #np.format_float_scientific(m, exp_digits=4)
    
    plt.plot(x_to_fit, y_to_fit)
    plt.plot(x_to_fit_ends, (x_to_fit_ends*m) + b, linestyle='--', marker='.')
    m = np.format_float_positional(m, precision=4)
    m_pm =np.format_float_positional(m_pm, precision=4)
    b = np.format_float_positional(b, precision=4)
    b_pm = np.format_float_positional(b_pm, precision=4)
    plt.plot([], [], ' ', label="$m= {} +/- {}$".format(m, m_pm))
    plt.plot([], [], ' ', label= "$b={} +/- {}$".format(b, b_pm))
    plt.xlabel(xlabel)
    plt.ylabel(ylabel)
    plt.legend(loc=0)
    #print("m= {} +/- {}".format(coefficients[0], np.sqrt(V[0][0])))
    #print("b: {} +/- {}".format(coefficients[1], np.sqrt(V[1][1])))
    
    return float(m), float(b)


$$ \frac{dx}{dt} = a x - b x^2 + x \epsilon(t) - \Gamma (t) $$
$$ < \epsilon(t) \epsilon(t') > = 2 D \delta(t-t') $$
$$ < \Gamma(t) \Gamma(t') > = 2 \alpha \delta(t-t') $$
$$ < \epsilon(t) \Gamma(t') > = 2 \lambda \sqrt{D \alpha} \delta(t-t') $$

In [1]:
def logistic_x(start=0.001,L=10**5,a=1,b=0.1,D=0,alpha=0,lamb=0,renew=True,plot=True):
    
    """
    Creates data using the logistic equation with an Ito method.
        dx/dt = ax - bx^2 + x \epsilon(t) - \Gamma(t)
    Uses notation from the paper
        Correlated noise in a logistic growth model.
    
    Parameters
    ----------
    start :
        Where the trajectory should start. Close to zero.
    L :
        How many data points should be created.
    a :
        Growth rate.
    b:
        Decay rate.
    D:
        epsilon = sqrt(2*D)
        Affects strength of multiplicative noise.
    alpha:
        Gamma = sqrt(2*alpha)
        Affects strength of additive noise.
    lamb (lambda):
        Denotes the correlation between epsilon and Gamma.
    renew: Bolean
        If TRUE then the trajectory will restart when crossing zero.
    plot: Bolean
        If TRUE the function will plot the trajectory.
        
    Returns
    ----------
    x : 
        Trajectory using logistic equation.

    """
    
    D = float(D)
    alpha = float(alpha)
    
    #Correlation occurs when lamb is greater than zero.
    if lamb > 0:
        choices_magnitude = [np.sqrt(2*D), np.sqrt(2*alpha)] #Gamma will be either of these values.
        Gamma_choice = random.choices(choices_magnitude, weights = [(lamb)*10 , (1-lamb)*10], k=L)
    else:
        Gamma_choice = np.sqrt(2*alpha) #Note this means that there will be no additive noise when alpha=0.
            
    epsilon = np.random.normal(loc=0, scale = np.sqrt(2*D), size=L) #Strength of muliplicative noise.
    
    #np.random.choice([1,-1],L) #Strength of muliplicative noise.
    
    Gamma = np.random.normal(loc=0.0, scale=Gamma_choice, size=L) #Gamma positive or negative.
    
    #np.random.choice([1,-1],L) #Gamma positive or negative.

    x = np.zeros(L) #create array of zeroes.
    
    x[0]=start

    for t in tqdm_notebook(range(1,L), desc='logistic_x', leave=False):
            x[t] = x[t-1] + a*x[t-1] - b*x[t-1]**2 + x[t-1]*epsilon[t-1] + Gamma[t-1]; #ITO
            if renew == True:
                if x[t-1] < 0:
                    x[t] = start #It's negative restart it at the start point.
    if plot == True:
        title = 'x'
        plt.title(title)
        plt.plot(x, label='L='+ str(L))
        plt.plot([], [], ' ', label= '$x_0$='+str(start))
        plt.plot([], [], ' ', label= '$D$='+str(f"{D:.3}"))
        plt.plot([], [], ' ', label= '$\\epsilon$='+str(f"{np.sqrt(2*D):.3}"))
        plt.plot([], [], ' ', label= '$\\alpha$='+str(f"{alpha:.3}"))
        plt.plot([], [], ' ', label= '$\\Gamma$='+str(f"{np.sqrt(2*alpha):.3}"))
        plt.plot([], [], ' ', label= '$a$='+str(a))
        plt.plot([], [], ' ', label= '$b$='+str(b))
        plt.plot([], [], ' ', label= '$a / b$='+str(a/b))
        plt.plot([], [], ' ', label= '$\\lambda$='+str(lamb))


        plt.xlabel('t')
        plt.ylabel('x')
        plt.legend(loc='upper right')
        plt.show()
    return x

In [2]:
def log_logistic(start=0.001,L=10**5,a=1,b=0.1,D=0,renew=True,plot=True):
    D = float(D)
    
    xi = np.sqrt(2*D)*np.random.normal(loc=0.0, scale=np.sqrt(1.0), size=L)
    #np.random.choice([1,-1],L)
    
    #Q = epsilon^2/2

    y = np.zeros(L)
    
    y[0]=start

    for t in tqdm(range(1,L)):
            y[t] = y[t-1] + a - b*np.exp(y[t-1]) + xi[t-1]
            if renew == True:
                if y[t-1] < 0:
                    y[t] = start #It's negative restart it at the start point.
            
    if plot == True:
        plt.title('y=ln(x)')
        plt.plot(y, label='L='+ str(L))
        plt.plot([], [], ' ', label= '$y_0$='+str(start))
        plt.plot([], [], ' ', label= '$D$='+str(f"{D:.3}"))
        plt.plot([], [], ' ', label= '$\\epsilon$='+str(f"{np.sqrt(2*D):.3}"))
        plt.plot([], [], ' ', label= '$a$='+str(a))
        plt.plot([], [], ' ', label= '$b$='+str(b))
        plt.plot([], [], ' ', label= '$\ln(a/b)$='+str(f"{np.log(a/b):.3}"))
        plt.xlabel('t')
        plt.ylabel('y')
        plt.legend(loc='upper right')
        plt.show()
    return y

## correleation_function
y_series is the series you want to analyze

correlation(i) = $\frac{\sum (y(t)-\log(10))(y(t+l)-\log(10)) }{\sum (y(t)-\log(10))^2}$

In [1]:
def correlation_function(l, y_series):
    
    #This is where we will store the results.
    correlation = np.zeros(len(l))
    
    m_y = np.mean(y_series)

    #Step through each window.
    for i in tqdm(range(len(l))):
        #store the correlation numerator here
        correlation_num = np.zeros(len(y_series)-l[i])
        #store the correlation denomonator here
        correlation_den = np.zeros(len(y_series)-l[i])
        #Step through the series
        #Make sure you don't overstep the last of the series
        for t in range(len(y_series)-l[i]):
            correlation_num[t] = (y_series[t]-m_y)*(y_series[t+l[i]]-m_y)
            correlation_den[t] = (y_series[t]-m_y)**2
        
        correlation[i] = sum(correlation_num)/sum(correlation_den)
        
    return correlation

In [ ]:
def survival_prob(bins, counts, plot,x_label,y_label):
        counts_total = sum(counts)
        counts_cnts = [counts_total]
        for bins_i in range(len(bins)):
            counts_total = counts_total - counts[bins_i]
            counts_cnts.append(counts_total)
        if plot == True:
            title = 'Survival Probability'
            plt.title(title)
            plt.plot(counts_cnts)
            plt.xlabel(x_label)
            plt.ylabel(y_label)
            #plt.legend(loc='upper right')
            plt.show()
            
        return list(range(len(counts_cnts))), counts_cnts

In [56]:
def test_function(L, y_0, beta, Q):
    
    epsilon = np.sqrt(Q)
    
    xi = epsilon*np.random.choice([1,-1],L)
    
    y = np.zeros(L)
    
    y[0] = y_0
    
    for t in tqdm(range(1,L)):
            y[t] = y[t-1]*(1 - beta) + xi[t-1]
    
    plt.title('$y[t] = y[t-1]*(1 - \\beta) + \\xi$')
    plt.plot(y)
    plt.plot([], [], ' ', label='L='+ str(L))
    plt.plot([], [], ' ', label= '$Q$='+str(Q))
    plt.plot([], [], ' ', label= '$\\epsilon$='+str(epsilon))
    plt.plot([], [], ' ', label= '$\\beta$='+str(beta))
    plt.plot([], [], ' ', label= '$y_0$='+str(y_0))
    plt.plot([], [], ' ', label= '$a$='+str(beta/Q))
    plt.xlabel('t')
    plt.ylabel('y')
    plt.legend()
    plt.show()
    return y

In [ ]:
def non_linear_fit(which, xdata, ydata, p0, start, stop, plot=True):
    
    """
    Fits power and exponential plots. Outputs result in log-log or log-lin plot.

    Parameters
    ----------
    which : 
        string
        select either "power" or "exponential"
    
    xdata :
        Data that is plotted on the x axis. Generally bins.
    
    ydata :
        Data that is plotted on the y axis. Generally counts.
        
    p0 :
        This is an array of guesses for the fit. [b, m] for y = mx+b
    
    start :
        Where in the data to start the plot. x-value.
        
    stop :
        Where in the data to stop the plot. x-value.
        
    plot :
        Boolean
        Should the program output a plot of the data and the fit.

    Returns
    ----------
    [m, b], [m_pm, b_pm]
    
    location :
        Location of crossings. This could also serve as a trajectory/
    """
    
    xdata = np.array(xdata)
    ydata = np.array(ydata)
    
    if which == "power_law":
        
        start = np.exp(start)
        stop = np.exp(stop)
        
        #find closes values
        start = np.abs(np.array(xdata) - start).argmin()
        stop = np.abs(np.array(xdata) - stop).argmin()
        
        xdata_to_fit = xdata[start:stop]
        ydata_to_fit = ydata[start:stop]
        
        #Function to calculate the power-law with constants a and b
        def power_law(x, a, b):
            return a*np.power(x, b)
        #Fit the power-law data
        pars, cov = curve_fit(power_law, xdata_to_fit, ydata_to_fit, p0, bounds=(-np.inf, np.inf))

        # Get the standard deviations of the parameters (square roots of the diagonal of the covariance)
        stdevs = np.sqrt(np.diag(cov))

        # Calculate the residuals
        res = ydata_to_fit - power_law(xdata_to_fit, *pars)
        
    elif which == "exponential":
        
        #find closes values
        start = np.abs(np.array(xdata) - start).argmin()
        stop = np.abs(np.array(xdata) - stop).argmin()
        
        xdata_to_fit = xdata[start:stop]
        ydata_to_fit = ydata[start:stop]
        
        #Function to calculate the exponential with constants a and b
        def exponential(x,a,b):
            return a*np.exp(b*x)
        
        #Fit the exponential data
        pars, cov = curve_fit(exponential, xdata_to_fit, ydata_to_fit, p0, bounds=(-np.inf, np.inf))
        
        # Get the standard deviations of the parameters (square roots of the # diagonal of the covariance)
        stdevs = np.sqrt(np.diag(cov))
        
        # Calculate the residuals
        res = ydata_to_fit - exponential(xdata_to_fit, *pars)
    
    m = pars[1]
    b = np.log(pars[0])
    m_pm = stdevs[1]
    b_pm = np.abs(1/pars[0])*stdevs[0]
    
    if plot == True:
    
        value_locations = np.where(ydata != 0)
        
        if which == "power_law":
            xdata_log = np.log(xdata[value_locations])
            ydata_log = np.log(ydata[value_locations])
            
            #find closes values
            start = np.abs(np.array(xdata_log) - start).argmin()
            stop = np.abs(np.array(xdata_log) - stop).argmin()
            
            #find the ends to draw the line
            x_to_fit_ends = np.array([np.log(xdata_to_fit[0]), np.log(xdata_to_fit[-1])])
            
            plt.plot(xdata_log, ydata_log)
            plt.xlabel("Log(x)")
            plt.ylabel("Log(y)")
        
        elif which == "exponential":
            xdata_n0 = xdata[value_locations]
            ydata_log = np.log(ydata[value_locations])
            
            #find the ends to draw the line
            x_to_fit_ends = np.array([xdata_to_fit[0], xdata_to_fit[-1]])
                        
            plt.plot(xdata_n0, ydata_log)
            plt.xlabel("x")
            plt.ylabel("Log(y)")
        
        plt.plot(x_to_fit_ends, (x_to_fit_ends)*m + b, linestyle='--', marker='.')
        m_str = np.format_float_positional(m, precision=4)
        m_pm_str =np.format_float_positional(m_pm, precision=4)
        b_str = np.format_float_positional(b, precision=4)
        b_pm_str = np.format_float_positional(b_pm, precision=4)
        plt.plot([], [], ' ', label="$m= {} +/- {}$".format(m_str, m_pm_str))
        plt.plot([], [], ' ', label= "$b={} +/- {}$".format(b_str, b_pm_str))
        
        plt.legend(loc=0)
        
        #print("m= {} +/- {}".format(coefficients[0], np.sqrt(V[0][0])))
        #print("b: {} +/- {}".format(coefficients[1], np.sqrt(V[1][1])))
        
    return [m, b], [m_pm, b_pm]

In [ ]:
def split_matrix_threshold(list_array, element, threshold):
    
    """
    Takes an array and seperates the list at negative values. Then rejoined into a matrix with an element.
    
    Parameters
    ----------
    
    list_array :
        Array that will be seperated at negative values.
    
    element :
        Element is what will be used to create the matrix. Usually '0' or 'numpy.NaN'
        
    threshold :
        What threshold to split the list_array into a matrix. Setting to 0 will split at negative values.
    
    Returns
    -------
    
    sep_traj:
        Numpy array with nan values. (matrix)
    """

    split_indicies = np.where(list_array<threshold)[0] #find indicies where negative values
        
    traj_length = np.diff(split_indicies, prepend=-1) #Find trajectory lengths.
    #Note that prepend=-1 to ensure that beginning will be counted.
        
    sep_traj = np.array_split(list_array,split_indicies+1)[:-1] #Separate trajectories using indicies.

    #Create a function that appends the element.
    def append_stuff(a, b):
        return np.append(a,np.full(b,element))

    #map the function on separated trajectory
    sep_traj = np.array(list(map(append_stuff, sep_traj, max(traj_length)-traj_length)))

    return sep_traj

In [ ]:
def matrix_diff_locations(x_l, y_l):
    """
    Takes indicies for crossings in a matrix and returns the differences in crossings in one array.
    
    Parameters
    ----------
    
    x_l : Row index for matrix of crossings.
    
    y_l : Column index for matrix of crossings.
    
    Returns
    -------
    
    taus:
        A single array filled with all differences based on the crossings.
    
    """
    #First make a matrix of NaN whose size is based on the largest occurence in index x_l.
    new_matrix = np.full((len(np.unique(x_t)),max(np.bincount(x_t))),np.NaN)
    
    #Step through each column in the new_matrix and each value in x_t.
    for i, x_i in zip(range(len(np.unique(x_t))), np.unique(x_t)):
        y_values = y_t[x_t == x_i] #Grab that y_l values using x_l.
        new_matrix[i,:len(y_values)] = y_values #Put these values in the new matrix.
        
        taus = np.diff(new_matrix).flatten() #Flatten into one matrix.

        taus = taus[True != np.isnan(taus)] #Get rid of NaN.
        
        return taus.astype(int) #Convert floats to integers.

In [ ]:
def matrix_neg_nan(list_array):
    
    """
    Takes an array and seperates the list at negative values. Then rejoined into a matrix with nan values.
    
    Parameters
    ----------
    
    list_array :
        Array that will be seperated at negative values.
    
    Returns
    -------
    
    sep_traj:
        Numpy array with nan values. (matrix)
    """
    

    neg_indicies = np.where(list_array<0)[0] #find indicies where negative values

    traj_length = np.diff(neg_indicies, prepend=-1) #

    sep_traj = np.array_split(test,neg_indicies+1)[:-1]

    def append_stuff(a, b):
        return np.append(a,np.full(b,np.NaN))

    sep_traj = np.array(list(map(append_stuff, sep_traj, max(traj_length)-traj_length)))

    return sep_traj